In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("archive/2015.csv")
data.head()

,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENUM,...,_PAREC1,_PASTAE1,_LMTACT1,_LMTWRK1,_LMTSCL1,_RFSEAT2,_RFSEAT3,_FLSHOT6,_PNEUMO2,_AIDTST3
0,1.0,1.0,b'01292015',b'01',b'29',b'2015',1200.0,2.015000e+09,2.015000e+09,1.0,...,4.0,2.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,1.0
1,1.0,1.0,b'01202015',b'01',b'20',b'2015',1100.0,2.015000e+09,2.015000e+09,1.0,...,2.0,2.0,3.0,3.0,4.0,2.0,2.0,NaN,NaN,2.0
2,1.0,1.0,b'02012015',b'02',b'01',b'2015',1200.0,2.015000e+09,2.015000e+09,1.0,...,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,NaN
3,1.0,1.0,b'01142015',b'01',b'14',b'2015',1100.0,2.015000e+09,2.015000e+09,1.0,...,4.0,2.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,9.0
4,1.0,1.0,b'01142015',b'01',b'14',b'2015',1100.0,2.015000e+09,2.015000e+09,1.0,...,4.0,2.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,1.0


Dataset notes:

https://www.cdc.gov/brfss/annual_data/2011/pdf/CODEBOOK11_LLCP.pdf 2011 codebook

https://www.cdc.gov/brfss/annual_data/2015/pdf/codebook15_llcp.pdf 2015 codebook

How many days unwell

Person, quesitons on:

    Blood pressure
    Asthma
    Cancer, etc.

Mental health questions too:

    Depression
    
Demographics / correlates:

    Age
    Veteran
    Married
    Children?
    Education
    Income
    Weight
    Height
    Sex (male/female)
    
    Smoked at least 100 cigarettes
    Eating fruit
    Beans
    Vegtables
    Exercise
    Alcohol
    Diabetes? (APPARENTLY THIS AFFECTS VISION??? COULD CHECK??)
    Multivitamin taking
    Sleep
    Work hours
    HIV?? (although maybe just tests and not if you have it)
    If you were touched sexually as a child
    If your child has asthma
    Child demographics
    

## Variables to clean:::
35-69

(More vegetable stuff above)

FVBEANS - How much do you eat beans?

FVGREEN - How many times did you eat dark green vegetables?

FVORANG - orange vegetables

VEGETAB1 - eat other vegtables

EXERANY2 - Exercise in last 30 days

EXRACT11 - type of physical activity (but there’s almost 100 options probably not good)

STRENGTH - How many times did strength to exercise?

Joints:

    LMTJOIN3 - Limited in activity cuz of joint pain	
    
    ARTHDIS2 - Does arthritis affect work?
    
    ARTHSOCL - Social Activities limited because of Joint Pain?	
    
    JOINPAIN - How bad is joint pain?
    
SEATBELT - How often wear seatbelt	

PREDIAB1 - Ever been told by doctor you were pre diabetes?

CAREGIV1 - Provide care to family in last 30 days?		

CRGVLNG1 - How long have you been caring for them?

CRGVHRS1 - How many hours do you provide care for them?

CIMEMLOS - Have you experienced confusion of memory loss that is getting worse?

DRADVISE - Doctor told you to reduce sodium intake

In [3]:
variables = ["FVBEANS", "FVGREEN", "FVORANG", "VEGETAB1", "EXERANY2", "EXEROFT1", "STRENGTH", "LMTJOIN3", "ARTHDIS2", 
            "JOINPAIN", "PREDIAB1", "CAREGIV1", "CRGVLNG1", "CRGVHRS1", "CIMEMLOS", "DRADVISE"]

data = data.loc[:, variables]
clean = data
data.head()


,FVBEANS,FVGREEN,FVORANG,VEGETAB1,EXERANY2,EXEROFT1,STRENGTH,LMTJOIN3,ARTHDIS2,JOINPAIN,PREDIAB1,CAREGIV1,CRGVLNG1,CRGVHRS1,CIMEMLOS,DRADVISE
0,320.0,310.0,305.0,101.0,2.0,NaN,888.0,1.0,1.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN
1,302.0,202.0,202.0,304.0,1.0,212.0,888.0,NaN,NaN,NaN,3.0,2.0,NaN,NaN,1.0,2.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN
3,555.0,301.0,301.0,201.0,2.0,NaN,888.0,1.0,1.0,8.0,3.0,2.0,NaN,NaN,1.0,2.0
4,203.0,204.0,310.0,320.0,2.0,NaN,888.0,1.0,1.0,7.0,3.0,2.0,NaN,NaN,2.0,2.0


In [4]:
#Numeric with 100 daily 200 weekly 300 monthly
numlist = ["FVBEANS", "FVGREEN", "FVORANG", "VEGETAB1"]
for var in numlist:
    clean[var] = np.where((data[var] >= 100) & (data[var] <= 199), 
                               np.mod(data[var], 100)*30, clean[var])
    clean[var] = np.where((data[var] >= 200) & (data[var] <= 299), 
                               np.mod(data[var], 200)*4, clean[var])
    clean[var] = np.where((data[var] >= 300) & (data[var] <= 399), 
                               np.mod(data[var], 300), clean[var])
    clean[var] = np.where(data[var] == 555, 0, clean[var])
    clean[var] = np.where(data[var] == 777, np.nan, clean[var])
    clean[var] = np.where(data[var] == 999, np.nan, clean[var])

#Numeric but 100+ is weekly 200+ is monthly
numlist2 = ["EXEROFT1", "STRENGTH"]
for var in numlist2:
    clean[var] = np.where((data[var] >= 100) & (data[var] <= 199), 
                               np.mod(data[var], 100)*4, clean[var])
    clean[var] = np.where((data[var] >= 200) & (data[var] <= 299), 
                               np.mod(data[var], 200), clean[var])
    clean[var] = np.where(data[var] == 777, np.nan, clean[var])
    clean[var] = np.where(data[var] == 888, 0, clean[var])
    clean[var] = np.where(data[var] == 999, np.nan, clean[var])
    
#Binary vars
binarylist = ["EXERANY2", "LMTJOIN3", "ARTHDIS2", "CIMEMLOS", "DRADVISE"]
for var in binarylist:
    clean[var] = np.where(data[var] == 2, 0, clean[var])
    clean[var] = np.where(data[var] == 7, np.nan, clean[var])
    clean[var] = np.where(data[var] == 9, np.nan, clean[var])
    
#1-10 vars
list1_10 = ["JOINPAIN"]
for var in list1_10:
    clean[var] = np.where(data[var] == 7, np.nan, clean[var])
    clean[var] = np.where(data[var] == 9, np.nan, clean[var])
    
#yes, yes during pregnancy, and no. Convert to binary
almost_binarylist = ["PREDIAB1"]
for var in almost_binarylist:
    clean[var] = np.where(data[var] == 2, 1, clean[var])
    clean[var] = np.where(data[var] == 3, 0, clean[var])
    clean[var] = np.where(data[var] == 7, np.nan, clean[var])
    clean[var] = np.where(data[var] == 9, np.nan, clean[var])
   

almost_binarylist2 = ["CAREGIV1"]
for var in almost_binarylist2:
    clean[var] = np.where(data[var] == 7, np.nan, clean[var])
    clean[var] = np.where(data[var] == 8, 1, clean[var]) #Died
    clean[var] = np.where(data[var] == 9, np.nan, clean[var])
    
CRGVLNG1_map = {1.0: '<30 days', 2.0: '1-6 months', 3.0: '0.5-2 years', 4.0: '2-5 years', 5.0: '5+ years'}
clean["CRGVLNG1"] = clean["CRGVLNG1"].replace(CRGVLNG1_map)
clean["CRGVLNG1"] = np.where(data["CRGVLNG1"] == 7, np.nan, clean["CRGVLNG1"])
clean["CRGVLNG1"] = np.where(data["CRGVLNG1"] == 9, np.nan, clean["CRGVLNG1"])

CRGVHRS1_map = {1.0: '0-8 hours/week', 2.0: '9-19 hours/week', 3.0: '20-39 hours/week', 4.0: '40+ hours/week'}
clean["CRGVHRS1"] = clean["CRGVHRS1"].replace(CRGVHRS1_map)
clean["CRGVHRS1"] = np.where(data["CRGVHRS1"] == 7, np.nan, clean["CRGVHRS1"])
clean["CRGVHRS1"] = np.where(data["CRGVHRS1"] == 9, np.nan, clean["CRGVHRS1"])

In [5]:
clean.head()

,FVBEANS,FVGREEN,FVORANG,VEGETAB1,EXERANY2,EXEROFT1,STRENGTH,LMTJOIN3,ARTHDIS2,JOINPAIN,PREDIAB1,CAREGIV1,CRGVLNG1,CRGVHRS1,CIMEMLOS,DRADVISE
0,20.0,10.0,5.0,30.0,0.0,NaN,0.0,1.0,1.0,6.0,0.0,NaN,NaN,NaN,NaN,NaN
1,2.0,8.0,8.0,4.0,1.0,12.0,0.0,NaN,NaN,NaN,0.0,2.0,NaN,NaN,1.0,0.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
3,0.0,1.0,1.0,4.0,0.0,NaN,0.0,1.0,1.0,8.0,0.0,2.0,NaN,NaN,1.0,0.0
4,12.0,16.0,10.0,20.0,0.0,NaN,0.0,1.0,1.0,NaN,0.0,2.0,NaN,NaN,0.0,0.0
